In [1]:
using Plots

In [2]:
plotly()

Plots.PlotlyBackend()

In [3]:
using Interact

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/611af8fe17b42a003eb07022de01433e0b5a0ed8-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/611af8fe17b42a003eb07022de01433e0b5a0ed8-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [4]:
Tc = 2/log(1+sqrt(2)) #TEMPERATURA CRÍTICA

2.269185314213022

In [5]:
N = 100
dE = zeros(N, N)
T = 2
S =[rand([-1,1]) for i in 1:N, j in 1:N];

In [6]:
#función de probabilidad en función de la energía y temperatura
prob(dE, T) = min(1, exp(-dE/T))

prob (generic function with 1 method)

In [7]:
promedio(M) = sum(i for i in M)/length(M)

promedio (generic function with 1 method)

In [8]:
#funciones para testear los límites de la malla
function fmas(x, L) #x = n+1
        if x == L
        xn = 1 # xn -> n+1
        else xn = x + 1
        end
    return xn
end

function fmenos(x, L) # x =  n-1
    if x == 1
       xn = L # xn -> n-1
    else xn = x-1
    end
    
    return xn
end

fmenos (generic function with 1 method)

In [9]:
d_E(ns, N) =  [-0.5*ns[n,m]*(ns[fmas(n,N),m] + ns[fmenos(n,N),m] + ns[n,fmenos(m,N)] + ns[n,fmas(m,N)]) for n in 1:N, m in 1:N]


d_E (generic function with 1 method)

In [10]:
C_e(N,T,dE) = (N/T^2)*(promedio(dE.^2) - promedio(dE)^2)

Ξ( N, T, s) = (promedio(s.^2)-promedio(s)^2)*N/T 

Ξ (generic function with 1 method)

##  Función maestra (Algoritmo Metrópolis)

In [11]:
function Metropolis(T, S, N, tf)  #recibe (Temperatura, S=Config. inicial de Spines, N= dim  de S, tf = número de pasos)

     ns = copy(S)
    
    for j in 1:tf
    
        (n,m) = rand(1:N,2) #escoge un spín al azar

        #Calculo la energía promedio de los vecinos
        delta_E = 2*ns[n,m]*( ns[fmas(n,N),m] + ns[fmenos(n,N),m] + ns[n,fmenos(m,N)] + ns[n, fmas(m,N)])
        # fmas y fmenos dan las condiciones a la frontera
    
        p = rand() #parametro que decide si cambiar de estado

        if p < prob(delta_E, T) #condicion para mover el spin o no
            
            ns[n,m] = -ns[n,m] #cambia signo del spin 
        end
        
       
    end
        dE =  d_E(ns, N) 
         E = promedio(dE)
         M = promedio(ns)
         C = C_e(N,T,dE) #calor específico
         X =  Ξ(N, T, ns)
    return ns, E, M, C, X  # Matriz evolucionada, Energía,  Magnetización por sitio, Calor específico
end
    

Metropolis (generic function with 1 method)

## Gráficas de Energía y Magnetización vs Temperatura

In [12]:
N = 20
t = 10^6   #tiempo (pasos metrópolis)
S =[rand([-1,1]) for i in 1:N, j in 1:N]

TT =  0:0.01:7 #rango de temperaturas

Energías = Float64[]
Magnetización = Float64[]
#Calore = Float64[]
                                #Iteramos sobre el tiempo
    for i in TT                       
        NS, E , M, C, X = Metropolis(i, S, N, t)    
        push!(Magnetización, M)
        push!(Energías, E)
        #push!(Calore, C)
    end

scatter(TT, Energías, label = "Energía", xlabel ="Temperatura")
scatter!(TT,Magnetización, label = "Magnetización")
#scatter!(TT, Calore,  label = "Calor específico")

## Gráficas de magnetización y energía promediados

In [13]:

N = 20
tf = N*N
p = 10^4      # p pasos de tf

prom_magn = Float64[]
prom_ener = Float64[]
prom_ce = Float64[]
prom_x = Float64[]

S =[rand([-1,1]) for i in 1:N, j in 1:N] #Matriz arbitraria de Spines (condición inicial)

TT = 0:0.01:6  #Rango de temperaturas

for T in TT  #recorre sobre la temperatura
    
    
    ns = S #condicion inicial
    E = promedio(d_E(ns, N)) #energía inicial

    Magnet = Float64[promedio(S)]
    energía = Float64[E]
    Calore = Float64[C_e(N,T,d_E(ns, N))]
    Suscept = Float64[Ξ(N, T, ns)]
    
    for i in 1:p
        ns, E, M, C, X = Metropolis(T, ns, N, tf) 
        push!(energía,copy(E))
        push!(Magnet, copy(M))
        push!(Calore, copy(C))
        push!(Suscept, copy(X))
    end
    
   m = promedio(Magnet[20^3:end]) #seleccionamos un rango de los arreglos donde notamos que ya hay "convergencia" para
   e = promedio(energía[20^3:end])      #todas las temperaturas 
   c = promedio(Calore[20^3:end])
   x = promedio(Suscept[20^3:end])
     push!(prom_magn, m) #guardamos los promedios en un arreglo
     push!(prom_ener, e)
     push!(prom_ce, c)
     push!(prom_x, x)

end
#return prom_magn, prom_ener

In [14]:
scatter(TT, prom_magn, label = "Magnetización promedio", xlabel = "Temperatura")
scatter!(TT, prom_ener, label = "Energía promedio")

In [15]:
scatter(TT, prom_ce, label ="Calor específico", xlabel = "Temperatura")
scatter!(TT, prom_x, label = "Susceptibilidad", ylim = (0,10))

## Evolución

In [16]:
T = 0.2
N = 50
tf = N*N
p = 500
S0 = [rand([-1,1]) for i in 1:N, j in 1:N]
S = S0
evolution = [copy(S0)]

for i in 1:p
 
        NS, E, M = Metropolis(T, S, N, tf) 
        S = NS
        push!(evolution, S)
end
tt = 0:tf:p*tf;

In [23]:
@manipulate for i in 1:length(tt)
    heatmap(evolution[i])
end

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:i=>Widget{:slider,Int64}(OrderedDict{Symbol,Any}(:changes=>Observable{Int64} with 1 listeners. Value:
0,:value=>Observable{Int64} with 2 listeners. Value:
501), Observable{Int64} with 2 listeners. Value:
501, Scope("knockout-component-562faf63-7fe9-43bb-ba19-289361d12183", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["i"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-left")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>1001,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-center")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-right")), 1)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"value"=>(Observable{Int64} with 2 listeners. Value:
501, nothing)), Set(String[]), nothing, Any["knockout"=>"C:\\Users\\Carolina\\.julia\\packages\\Knockout\\JIqpG\\src\\..\\assets\\knockout.js", "knockout_punches"=>"C:\\Users\\Carolina\\.julia\\packages\\Knockout\\JIqpG\\src\\..\\assets\\knockout_punches.js", "C:\\Users\\Carolina\\.julia\\packages\\InteractBase\\3SqBl\\src\\..\\assets\\all.js", "C:\\Users\\Carolina\\.julia\\packages\\InteractBase\\3SqBl\\src\\..\\assets\\style.css", "C:\\Users\\Carolina\\.julia\\packages\\InteractBulma\\Ohu5Y\\src\\..\\assets\\main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"changes\\\":0,\\\"value\\\":501}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"knockout-component-562faf63-7fe9-43bb-ba19-289361d12183\",\"id\":\"ob_17\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"changes\"]=false}),self),this[\"value\"].subscribe((function (val){!(this.valueFromJulia[\"value\"]) ? (WebIO.setval({\"name\":\"value\",\"scope\":\"knockout-component-562faf63-7fe9-43bb-ba19-289361d12183\",\"id\":\"ob_16\",\"type\":\"observable\"},val)) : undefined; return this

In [18]:
[(-1)^(i+j) for i in 1:10, j in 1:10 ];


In [19]:
T1 = 0.2
N = 50
tf = N*N
p = 500
S1 = [(-1)^(i+j) for i in 1:N, j in 1:N ]
S = S1
evolution1 = [copy(S)]

for i in 1:p
 
        NS, E, M = Metropolis(T, S, N, tf) 
        S = NS
        push!(evolution1, S)
end
tt = 0:tf:p*tf;

In [20]:
@manipulate for i in 1:length(tt)
    heatmap(evolution1[i])
end

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:i=>Widget{:slider,Int64}(OrderedDict{Symbol,Any}(:changes=>Observable{Int64} with 1 listeners. Value:
0,:value=>Observable{Int64} with 2 listeners. Value:
251), Observable{Int64} with 2 listeners. Value:
251, Scope("knockout-component-502396a4-7494-4f01-a934-d1c206f63952", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["i"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-left")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>501,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-center")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-right")), 1)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"value"=>(Observable{Int64} with 2 listeners. Value:
251, nothing)), Set(String[]), nothing, Any["knockout"=>"C:\\Users\\Carolina\\.julia\\packages\\Knockout\\JIqpG\\src\\..\\assets\\knockout.js", "knockout_punches"=>"C:\\Users\\Carolina\\.julia\\packages\\Knockout\\JIqpG\\src\\..\\assets\\knockout_punches.js", "C:\\Users\\Carolina\\.julia\\packages\\InteractBase\\3SqBl\\src\\..\\assets\\all.js", "C:\\Users\\Carolina\\.julia\\packages\\InteractBase\\3SqBl\\src\\..\\assets\\style.css", "C:\\Users\\Carolina\\.julia\\packages\\InteractBulma\\Ohu5Y\\src\\..\\assets\\main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"changes\\\":0,\\\"value\\\":251}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"knockout-component-502396a4-7494-4f01-a934-d1c206f63952\",\"id\":\"ob_07\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"changes\"]=false}),self),this[\"value\"].subscribe((function (val){!(this.valueFromJulia[\"value\"]) ? (WebIO.setval({\"name\":\"value\",\"scope\":\"knockout-component-502396a4-7494-4f01-a934-d1c206f63952\",\"id\":\"ob_06\",\"type\":\"observable\"},val)) : undefined; return this.

In [21]:
T1 = 4
N = 50
tf = N*N
p = 1000
function f(i,j) 
    if i < N/2 +1 # || j < N/2 +1
         return -1
    else
        return 1
    end
end
S1 = [f(i,j) for i in 1:N, j in 1:N ]
S = S1
evolution1 = [copy(S)]

for i in 1:p
 
        NS, E, M, C, x = Metropolis(T1, S, N, tf) 
        S = NS
        push!(evolution1, S)
end
tt = 0:tf:p*tf;

In [22]:
@manipulate for i in 1:length(tt)
    heatmap(evolution1[i])
end

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:i=>Widget{:slider,Int64}(OrderedDict{Symbol,Any}(:changes=>Observable{Int64} with 1 listeners. Value:
0,:value=>Observable{Int64} with 2 listeners. Value:
501), Observable{Int64} with 2 listeners. Value:
501, Scope("knockout-component-f2ea6d61-0503-46c4-95b9-c37ded27ed1e", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["i"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-left")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>1001,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-center")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-right")), 1)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"value"=>(Observable{Int64} with 2 listeners. Value:
501, nothing)), Set(String[]), nothing, Any["knockout"=>"C:\\Users\\Carolina\\.julia\\packages\\Knockout\\JIqpG\\src\\..\\assets\\knockout.js", "knockout_punches"=>"C:\\Users\\Carolina\\.julia\\packages\\Knockout\\JIqpG\\src\\..\\assets\\knockout_punches.js", "C:\\Users\\Carolina\\.julia\\packages\\InteractBase\\3SqBl\\src\\..\\assets\\all.js", "C:\\Users\\Carolina\\.julia\\packages\\InteractBase\\3SqBl\\src\\..\\assets\\style.css", "C:\\Users\\Carolina\\.julia\\packages\\InteractBulma\\Ohu5Y\\src\\..\\assets\\main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"changes\\\":0,\\\"value\\\":501}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"knockout-component-f2ea6d61-0503-46c4-95b9-c37ded27ed1e\",\"id\":\"ob_12\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"changes\"]=false}),self),this[\"value\"].subscribe((function (val){!(this.valueFromJulia[\"value\"]) ? (WebIO.setval({\"name\":\"value\",\"scope\":\"knockout-component-f2ea6d61-0503-46c4-95b9-c37ded27ed1e\",\"id\":\"ob_11\",\"type\":\"observable\"},val)) : undefined; return this

In [24]:
function giro(i, j)

    if i==j
    return  -1
    
    else
    return 1  
    end
end

giro (generic function with 1 method)

In [26]:
N = 20
tf = N*N
p = 10^4
d_E(ns) =  [-0.5*ns[n,m]*(ns[fmas(n,N),m] + ns[fmenos(n,N),m] + ns[n,fmenos(m,N)] + ns[n,fmas(m,N)]) for n in 1:N, m in 1:N]
prom_magn = Float64[]
prom_ener = Float64[]
calor_esp = Float64[]
sucep = Float64[]
TT = 0:0.1:5
for T in TT
    S = [giro(i, j) for i in 1:N , j in 1:N ] 
    ns = S #condicion inicial
    E = promedio(d_E(ns)) #energía inicial

    Magnet = Float64[promedio(S)]
    energía = Float64[E]
    for i in 1:p
        ns, E, M = Metropolis(T, ns, N, tf) 
        push!(energía,copy(E))
        push!(Magnet, copy(M))
    end
   m = promedio(Magnet[10^3:end]) #seleccionamos un rango de los arreglos donde notamos que ya hay "convergencia" para
   e = promedio(energía[10^3:end])  #todas las temperaturas 
   e_2 = promedio(energía[10^3:end].^2)  
   m_2 = promedio(Magnet[10^3:end].^2)  
   c = (N/(T^2))*(e_2 - e^2)
   x = (1/(T))*(m_2 - m^2) 
     push!(prom_magn, m) #guardamos los promedios en un arreglo
     push!(prom_ener, e)
     push!(calor_esp, c) 
     push!(sucep, x)
end

In [28]:
plot([2.26, 2.26 ],[0 , 0.2], xlabel="temperatura", label = "Tc = 2.269")
scatter!(TT, calor_esp, m=:o, label="calor específico")
scatter!(TT, sucep, m=:o,  label="suceptibilidad magnética")